In [159]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import winsound
import difflib
import re

## Construct URL

In [62]:
movie_name = "SpiderMan No Way home"
location = "Mumbai"
date = "16/12/2021" # DD/MM/YYYY
movie_code = "ET00319080" # Can be obtained from bookmyshow
cinema_type = "Carnival"

In [63]:
date_formatted = "".join(date.split("/")[::-1])
mov_name_formatted = "-".join(movie_name.lower().split())
base_url = "https://in.bookmyshow.com"
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36"}
url = base_url + f"/buytickets/{mov_name_formatted}-{location.lower()}/movie-{location.lower()}-{movie_code}-MT/{date_formatted}"
url

'https://in.bookmyshow.com/buytickets/spiderman-no-way-home-mumbai/movie-mumbai-ET00319080-MT/20211216'

## Checking if booking has started

In [64]:
r = requests.get("https://in.bookmyshow.com/buytickets/antim-the-final-truth-mumbai/movie-mumbai-ET00303116-MT/20211209", headers=headers)
r.url

'https://in.bookmyshow.com/buytickets/antim-the-final-truth-mumbai/movie-mumbai-ET00303116-MT/20211210'

In [65]:
# if r.url == url:
soup = BeautifulSoup(r.content, "html.parser")
venue_list = soup.find("ul", {"id": "venuelist"})
name = venue_list.find_all("li")[0].text

## Getting the right venue

In [66]:
translator = str.maketrans({chr(10): '', chr(9): ''})
name.translate(translator)

'Carnival: Sangam, AndheriINFOM-TicketF&B02:30 PMCancellation Available '

In [67]:
venues = venue_list.find_all("li")
for venue in venues:
    venue_name = venue.text.translate(translator)
    if venue_name.split()[0].rstrip(":").lower() == cinema_type.lower():
        print(base_url + venue.find("a", {"class": "__venue-name"})["href"])
    #print(venue)

https://in.bookmyshow.com/buytickets/carnival-sangam-andheri/cinema-mumbai-BISM-MT/20211210
https://in.bookmyshow.com/buytickets/carnival-imax-wadala/cinema-mumbai-ADWD-MT/20211210


## Play sound

In [69]:
freq = 100
dur = 50
for i in range(0, 20):    
    winsound.Beep(freq, dur)    
    freq+= 100
    dur+= 50

## Basic Main function 

In [70]:
def main():
    movie_name = "Spider Man No Way home"
    location = "Mumbai"
    date = "16/12/2021" # DD/MM/YYYY
    movie_code = "ET00319080" # Can be obtained from bookmyshow dynamic but ok
    cinema_type = "Carnival"
    
    date_formatted = "".join(date.split("/")[::-1])
    mov_name_formatted = "-".join(movie_name.lower().split())
    base_url = "https://in.bookmyshow.com"
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36"}
    url = base_url + f"/buytickets/{mov_name_formatted}-{location.lower()}/movie-{location.lower()}-{movie_code}-MT/{date_formatted}"
    
    it = 0
    while True:
        it += 1
        r = requests.get(url, headers=headers)
        if r.url == url:
            freq = 100
            dur = 50
            for i in range(0, 20):
                winsound.Beep(freq, dur)
                freq += 100
                dur += 50
            soup = BeautifulSoup(r.content, "html.parser")
            venue_list = soup.find("ul", {"id": "venuelist"}).find_all("li")

            translator = str.maketrans({chr(10): '', chr(9): ''})
            # name.translate(translator)

            for venue in venue_list:
                venue_name = venue.text.translate(translator)
                if venue_name.split()[0].rstrip(":").lower() == cinema_type.lower():
                    print(base_url + venue.find("a", {"class": "__venue-name"})["href"])
        print("Iteration", it)
        sleep(15)


In [71]:
main()

Iteration 1


KeyboardInterrupt: 

## Dynamically getting the movie code

In [72]:
url = base_url + f"/explore/upcoming-movies-{location}?referrerBase=movies"
print(url)

https://in.bookmyshow.com/explore/upcoming-movies-Mumbai?referrerBase=movies


In [73]:
r = requests.get(url)

In [74]:
soup = BeautifulSoup(r.content, "html.parser")
temp = soup.find_all("a", {"class": "commonStyles__LinkWrapper-sc-133848s-11"})[1]
temp_link = temp["href"]

In [76]:
temp_link.split("/")[-2]

'spiderman-no-way-home'

In [86]:
movies_list = soup.find_all("a", {"class": "commonStyles__LinkWrapper-sc-133848s-11"})
for movies in  movies_list:
    try:
        link = movies["href"].split("/")
        if link[-2] == mov_name_formatted:
            print(link[-1])
    except:
        pass

ET00310790
17


In [175]:
def get_movie_code(base_url, location, mov_name_formatted):
    print("Getting movie code...")
    url = base_url + f"/explore/upcoming-movies-{location}?referrerBase=movies"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    movies_list = soup.find_all("a", {"class": "commonStyles__LinkWrapper-sc-133848s-11"})
    res = {}
    for movies in  movies_list:
        link = movies["href"].split("/")
        if len(link) > 1:
            res[link[-2]] = link[-1]
            if link[-2] == mov_name_formatted:
                return link[-1]

    print("Movie code not found! Getting closest matches....")
    matches = difflib.get_close_matches(mov_name_formatted, res.keys(), cutoff=0.8)
    if len(matches) > 0:
        print("Closet matches:")
        for cnt, item in enumerate(matches):
            print(f"{cnt} | {item} | {res[item]}")
            
    ans = input("Enter index number of the closest match if the right movie is found(Y/N):").lower()
    if ans == "y":
        idx = int(input("Enter the index: "))
        return res[matches[idx]]
    elif ans == "n":            
        # Example ET00315808
        while True:
            code = input("Enter the code of the movie manually(ET00XXXXXX): ").rstrip().lstrip()
            if bool(re.match(r"ET00[0-9]{6}", code)):
                return code
            print("Not valid code")


In [176]:
get_movie_code(base_url, location, mov_name_formatted)

Movie code not found! Getting closest matches....
Closet matches:
0 | spiderman-no-way-home | ET00310790
Enter index number of the closest match if the right movie is found(Y/N):Y
Enter the index: 0


'ET00310790'

## Getting details

In [170]:
def get_details():
    movie_name = input("Enter Movie Name\n- Only Spaces no Special Characters\n")
    location = input("Enter City\n- Enter Popular cities that appear on bookmyshow\
                     \n- Enter full forms, example: National Captial Region instead of NCR")
    date = input("Enter date when movie is going to be released\n- Enter a future date\n- Format: DD/MM/YYYY strictly") # DD/MM/YYYY
    cinema_type = input("Enter theater type\n- Example: INOX, Cinepolis, Carnival, Gold, etc")
    
    date_formatted = "".join(date.split("/")[::-1])
    mov_name_formatted = "-".join(movie_name.lower().split())
    movie_code = get_movie_code(base_url, location, mov_name_formatted)

False